### Manipulation of Matrix Data - Part III

GDAL - Part III
Steps needed to manipulate images:

import libraries (GDAL, NumPy, etc)
set correct path of raster files
open a dataset for each file
check metadata compatibility
get the bands of each raster
convert bands to arrays in NumPy format
manipulate digital numbers present in matrices
(if necessary) save the information to a new raster file

In [1]:
# Import libraries
from osgeo import gdal
import numpy as np
from matplotlib import pyplot as plt

# Import constants
from gdalconst import *

# Report the use of exceptions
gdal.UseExceptions()

# library of system-related functions
# sys: System-specific parameters and functions
import sys

### Exercise - Open two images (LC08_20181120_band_4.tif and LC08_20181120_band_5.tif), perform band arithmetic (use NDVI) and apply slicing (pixels> 0.5)

In [16]:
#import earthpy.spatial as es
import os
from glob import glob
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [17]:
file_path= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/"

In [19]:
# open 2 images with 1 band each
#filename_LC08_20181120_band_4= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B*[4-5]*.TIF" # red
#filename_LC08_20181120_band_5= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B5.TIF" # nir
filename_LC08_20181120_band= glob("C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B*[4-5]*.TIF") # red

filename_LC08_20181120_band.sort()
LC08_20181120_band_stack, meta = es.stack(filename_LC08_20181120_band, nodata=-9999)

In [22]:
# open 2 images with 1 band each
#filename_LC08_20181120_band_4= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B*[4-5]*.TIF" # red
#filename_LC08_20181120_band_5= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B5.TIF" # nir
#filename_LC08_20181120_band= "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_L1TP_169056_20181120_20181129_01_T1_B*[4-5]*.TIF" # red

#filename_LC08_20181120_band.sort()
#LC08_20181120_band_stack, meta = es.stack(filename_LC08_20181120_band, nodata=-9999)
#filename_LC08_20181120_band_4.sort()
#LC08_20181120_band_4_stack, meta = es.stack(filename_LC08_20181120_band_4, nodata=-9999)

#filename_LC08_20181120_band_5.sort()
#LC08_20181120_band_5_stack, meta = es.stack(filename_LC08_20181120_band_5, nodata=-9999)
try:
    dataset_LC08_20181120_band_4 = gdal.Open(filename_LC08_20181120_band_4, GA_ReadOnly) 
    dataset_LC08_20181120_band_5 = gdal.Open(filename_LC08_20181120_band_5, GA_ReadOnly) 
except:
    print ("Error opening a file!")

# all images have a band each
LC08_20181120_band_4 = dataset_LC08_20181120_band_4.GetRasterBand(1)
LC08_20181120_band_5 = dataset_LC08_20181120_band_5.GetRasterBand(1)

# to perform band calculations, we use the numpy matrix conversion
numpy_LC08_20181120_band_4 = LC08_20181120_band_4.ReadAsArray()
numpy_LC08_20181120_band_5 = LC08_20181120_band_5.ReadAsArray()

# create vegetation index band and apply slicing
NDVI = es.normalized_diff(numpy_LC08_20181120_band_5, numpy_LC08_20181120_band_4)

#numpy_LC08_20181120_ndvi = (numpy_LC08_20181120_band_5 - numpy_LC08_20181120_band_4) / \
                    #(numpy_LC08_20181120_band_5 + numpy_LC08_20181120_band_4)
numpy_output = numpy_LC08_20181120_ndvi > 0.25

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in greater


### Saving a raster
In this example, let's open two bands, perform an arithmetic (NDVI) followed by a slice, and save the result to a GeoTIFF file. To save a file with images, you must create a new dataset, enter all metadata related to the geographic context (projection system, geographic boundary, etc.) as well as the number of bands, number of rows and columns.

In [26]:
# get metadata
rows = dataset_LC08_20181120_band_4.RasterYSize
columns = dataset_LC08_20181120_band_4.RasterXSize
band = dataset_LC08_20181120_band_4.RasterCount

# save band to GeoTIFF file
# set filename
filename_output = "C:/Users/mac/Documents/Lake_Abaya_Chamo/LC08_169056_20181120/LC08_L1TP_169056_20181120_20181129_01_T1.tar/LC08_20181120-ndvi-threshold.tif"
# set driver
driver = gdal.GetDriverByName ('GTiff')
# copy existing tape data type
data_type = LC08_20181120_band_4.DataType
# create new dataset
dataset_output = driver.Create (filename_output, columns, rows, band, data_type)
# copy spatial information from existing band
dataset_output.SetGeoTransform (dataset_LC08_20181120_band_4.GetGeoTransform ())
# copy projection information
dataset_output.SetProjection (dataset_LC08_20181120_band_4.GetProjectionRef ())
# write NumPy array data to band
dataset_output.GetRasterBand (1) .WriteArray (numpy_output)
# save values
dataset_output.FlushCache ()
# close dataset
dataset_output = None

Exercise - Create a save_band function to save a raster of a band, having as parameters a NumPy array with pixels, the filename GeoTIFF, and a reference dataset (for copying metadata)

In [27]:
def save_band (pixel_array, filename, reference_data):
    # get metadata
    rows = dataset_de_referencia.RasterYSize
    columns = dataset_de_referencia.RasterXSize
    bands = 1
    # set driver
    driver = gdal.GetDriverByName ('GTiff')
    # copy existing tape data type
    data_type = reference_data.GetRasterBand (1) .DataType
    # create new dataset
    dataset_output = driver.Create (filename, columns, rows, bands, data_type)
    # copy spatial information from existing band
    dataset_output.SetGeoTransform (dataset_de_referencia.GetGeoTransform ())
    # copy projection information
    dataset_output.SetProjection (dataset_de_referencia.GetProjectionRef ())
    # write NumPy array data to band
    dataset_output.GetRasterBand (1) .WriteArray (pixel_array)
    # save values
    dataset_output.FlushCache ()
    # close dataset
    dataset_output = None


Exercise - Given two 1 band rasters each (1 thematic map, 1 reference map), calculate the hit ratio of the thematic map classification and save a GeoTIFF file containing a map of agreement between the images.

In [ ]:
# open images
filename_reference = "images / referencia_area_urbana.tif"
filename_classification = "images / classification_area_urbana.tif"

try:
    dataset_reference = gdal.Open (filename_reference, GA_ReadOnly)
    dataset_classification = gdal.Open (filename_classification, GA_ReadOnly)
except:
    print ("Error opening file!")

# check metadata compatibility
if (dataset_reference.GetProjectionRef ()! = dataset_classification.GetProjectionRef ()):
    print ("Different Reference Systems")
elif (dataset_reference.GetGeoTransform ()! = dataset_classification.GetGeoTransform ()):
    print ("Different spatial metadata")
else:
    # get metadata
    rows = dataset_reference.RasterYSize
    columns = dataset_reference.RasterXSize

    # get the bands
    band_reference = dataset_reference.GetRasterBand (1)
    band_classification = dataset_classification.GetRasterBand (1)

    # generate pixel arrays
    numpy_reference = band_reference.ReadAsArray ()
    numpy_classification = band_classification.ReadAsArray ()

    # generate comparison matrix
    numpy_comparison = (numpy_reference == numpy_classification)
    accuracy = 100 * numpy_comparison.sum () / (rows * columns)

    # plot results
    plt.figure (figsize = (15, 4))

    plt.subplot (131)
    plt.imshow (numpy_reference)
    plt.title ('Reference Image')

    plt.subplot (132)
    plt.imshow (numpy_classification)
    plt.title ('Classified Image')

    plt.subplot (133)
    plt.imshow (numpy_comparison)
    plt.title ('Comparison Image,' + str (accuracy) + '% Accuracy')

    plt.show ()

# save image
filename = "images / comparison.tif"
save_band (numpy_comparison, filename, dataset_reference)

# close images
dataset_reference = None
dataset_classification = None

Exercise - Given 2 images (raster_target and raster_entry), find the location of the raster_target image inside the raster_entry image and plot the result (raster_entry with an overlapping x in the location)

In [ ]:

# define where the images are
raster_entrada = 'images / american-anticipation-audience-163368.jpg'
raster_alvo = 'images / target_2.jpg'

# generate gdal datasets
dataset_entrada = gdal.Open (raster_entrada, GA_ReadOnly)
target dataset = gdal.Open (target raster, GA_ReadOnly)

# get the bands
input_band = input_set.GetRasterBand (1) .ReadAsArray ()
target_band = target_data.GetRasterBand (1) .ReadAsArray ()

# get input image metadata
Input_lines = input_data.RasterYSize
Input_columns = input_ dataset.RasterXSize

# get target image metadata
Target_lines = target_data.RasterYSize
Target_columns = target_details.RasterXSize

# find target row / column in input image
row = 0
column = 0
# create variable to store the most similar region
highest_similar = 0
for r in range (In_Rows - Target_lines):
  for c in range (Input_Columns - Target_Columns):
    # create cropping window of same size as target_band
    window = input_band [r: r + Target_lines, c: c + Target_columns]
    # compare how many pixels are equal between the two windows
    equal pixels = (target band == window)
    sum = equal_ pixels.sum ()
    if (sum> major_similar):
      line = r
      column = c
      greater_similar = summation

# show result
plt.imshow (target_band, cmap = 'gray')
plt.title ('Target Image')

plt.figure (figsize = (16,9))
plt.imshow (bandwidth, cmap = 'gray', aspect = 'auto')
plt.title ('Input Image')
plt.plot (column + TargetColumns / 2, row + TargetColumns / 2, 'bx')

plt.show ()

# close the files
dataset_entrada = None
dataset_alvo = None